Install qlatent is required

In [1]:
!pip install qlatent

In [1]:
import importlib
from tqdm.auto import tqdm
from qlatent.qmnli.qmnli import *
from qlatent.qmnli.qmnli import _QMNLI, QMNLI
# importlib.reload(_QMNLI)

import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

0


In [2]:
# softmax_files = [False, True]
# softmax_files = [True]

def split_question(Q, index, scales, softmax, filters):
  result = []
  for s in scales:
    q = QCACHE(Q(index=index, scale=s))
    for sf in softmax:
      for f in filters:
        if sf:            
            qsf = QSOFTMAX(q,dim=[index[0], s])
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print((index, s),sf,f)
            result.append(qsf_f)
            
#             qsf = QSOFTMAX(q,dim=s)
#             qsf_f = QFILTER(qsf,filters[f],filtername=f)
#             print(s,sf,f)
#             result.append(qsf_f)
            
#             qsf = QSOFTMAX(q,dim=index[0])
#             qsf_f = QFILTER(qsf,filters[f],filtername=f)
#             print(index[0],sf,f)
#             result.append(qsf_f)
        else:
            qsf = QPASS(q,descupdate={'softmax':''})
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print(s,sf,f)
            result.append(qsf_f)
  return result

def dict_pos_neg(pos, neg, w):
  return dict(dict_same_weight(1.0*w/len(pos),pos), **dict_same_weight(-1.0*w/len(neg),neg))

def print_permutations(q):
#     for q in Q1s:
    W = q._pdf['W']
    print(q._descriptor)
    for i, (kmap, w) in enumerate(zip(q._keywords_map, W)):
        context = q._context_template.format_map(kmap)
        answer = q._answer_template.format_map(kmap)
#         sexisem_score = sexisem_classifier(context.strip('.') + ' ' +answer)
        print(f'{i}.',context ,'->', answer, w)
#     break


frequency_weights:SCALE = {
    'never':-4,
    'very rarely':-3,
    'seldom':-2,
    'rarely':-2,
    'frequently':2,
    'often':2,
    'very frequently':3,
    'always':4,    
}
    
    
binary_frequency_weights = {k: 1 if v > 0 else -1 for k, v in frequency_weights.items()}
rbinary_frequency_weights = {k: -1 if v > 0 else 1 for k, v in frequency_weights.items()}
frequency_pos = [k for k, v in frequency_weights.items() if v > 0]
frequency_neg = [k for k, v in frequency_weights.items() if v < 0]

In [3]:
# p = 'valhalla/distilbart-mnli-12-6'
p = "typeform/distilbert-base-uncased-mnli"
mnli = pipeline("zero-shot-classification",device=device, model=p)
mnli.model_identifier = p


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


# ASI

In [5]:
from qpsychometric.social_biases.ambivalent_sexism_inventory.asi_qmnli import asi_bg_qmnli, asi_bi_qmnli, asi_bp_qmnli, asi_h_qmnli

In [6]:
for Q in asi_h_qmnli:
    Qs = split_question(Q,
                        index=Q.q_index,
                        scales=[Q.q_scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
2
Query time: 0.9372756481170654
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.050073776920423065
Internal consistency (silhouette, correlation) for unfiltered: 0.5499349236488342
Internal consistency (Calinski&Harabasz)  for unfiltered: 6.838287016655492
Internal consistency (Davies&Bouldin) for unfiltered: 0.6230203322544302


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qlatent\qabstract\qabstract.py:365: FutureWarning: The provided callable <function mean at 0x000001DE345D2D40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
improper,0.0002,0.0910,0.1302,0.0926,0.1599,0.1425,0.1864,0.1971
prejudiced,0.0022,0.0922,0.1218,0.0589,0.1699,0.1659,0.1887,0.2004
sexist,0.0568,0.0955,0.1150,0.1091,0.1458,0.1369,0.1736,0.1673
legitimate,0.0076,0.2342,0.1900,0.2329,0.1174,0.1566,0.0516,0.0098
reasonable,0.3000,0.1558,0.1160,0.1525,0.0701,0.0814,0.0554,0.0688
rightful,0.3088,0.1425,0.1145,0.1570,0.0842,0.0868,0.0561,0.0501


(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
4
Query time: 0.6499640941619873
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.105381477383465
Internal consistency (silhouette, correlation) for unfiltered: 0.8295049667358398
Internal consistency (Calinski&Harabasz)  for unfiltered: 80.00162251307546
Internal consistency (Davies&Bouldin) for unfiltered: 0.19556112058670908


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qlatent\qabstract\qabstract.py:365: FutureWarning: The provided callable <function mean at 0x000001DE345D2D40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


...

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qlatent\qabstract\qabstract.py:365: FutureWarning: The provided callable <function mean at 0x000001DE345D2D40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
improper,0.0001,0.0869,0.1225,0.0528,0.1824,0.1483,0.1853,0.2219
prejudiced,0.0001,0.0812,0.1130,0.0424,0.1850,0.1610,0.1885,0.2288
sexist,0.0001,0.0192,0.0189,0.0021,0.2360,0.2071,0.2425,0.2741
legitimate,0.1184,0.2320,0.2059,0.2616,0.0334,0.1045,0.0433,0.0008
reasonable,0.3245,0.1563,0.1456,0.2012,0.0484,0.0691,0.0481,0.0068
rightful,0.2566,0.1881,0.1569,0.2050,0.0601,0.0723,0.0443,0.0166


(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
21
Query time: 0.7565691471099854
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.09992759535051239
Internal consistency (silhouette, correlation) for unfiltered: 0.7969951033592224
Internal consistency (Calinski&Harabasz)  for unfiltered: 62.399526199575554
Internal consistency (Davies&Bouldin) for unfiltered: 0.24026864714876553


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qlatent\qabstract\qabstract.py:365: FutureWarning: The provided callable <function mean at 0x000001DE345D2D40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
improper,0.0001,0.0689,0.1099,0.1008,0.1846,0.1580,0.1824,0.1955
prejudiced,0.0000,0.0002,0.0430,0.0001,0.2333,0.2292,0.2404,0.2539
sexist,0.0001,0.0006,0.0367,0.0003,0.2426,0.2350,0.2471,0.2376
legitimate,0.2596,0.2763,0.2125,0.2502,0.0002,0.0010,0.0002,0.0001
reasonable,0.3272,0.2278,0.2003,0.2445,0.0000,0.0001,0.0000,0.0000
rightful,0.2468,0.2359,0.1818,0.2054,0.0314,0.0768,0.0215,0.0003


# SD3

In [4]:
from qpsychometric.personality_traits.short_dark_triad.sd3_qmnli import sd3_qmnli

In [7]:
for Q in tqdm(sd3_qmnli):
    Qs = split_question(Q,
                        index=Q.q_index,
                        scales=[Q.q_scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

  0%|          | 0/27 [00:00<?, ?it/s]

(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
1
Query time: 0.053418636322021484
Mean score unfiltered [-2.0..2.0]: -0.0914825655927416
Internal consistency (silhouette, correlation) for unfiltered: 0.3925871478952264
Internal consistency (Calinski&Harabasz)  for unfiltered: 2.3965247843812887
Internal consistency (Davies&Bouldin) for unfiltered: 0.8979161375353252


index = ['emotion']
{'emotion', 'intensifier'} {'intensifier'} {'emotion'}
[]


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
good,0.0010,0.1728,0.1177,0.0991,0.1336,0.1476,0.1175,0.2108
kind,0.0008,0.0248,0.0072,0.0017,0.2328,0.2179,0.2357,0.2791
hostil,0.0226,0.1571,0.2119,0.2291,0.1083,0.0870,0.1096,0.0744
mean,0.3953,0.1255,0.1320,0.1351,0.0616,0.0793,0.0711,0.0001


(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
4
Query time: 0.023447275161743164
Mean score unfiltered [-2.0..2.0]: -0.08156496868468821
Internal consistency (silhouette, correlation) for unfiltered: 0.7956193348832186
Internal consistency (Calinski&Harabasz)  for unfiltered: 13.449484260325457
Internal consistency (Davies&Bouldin) for unfiltered: 0.3460800956761326


index = ['emotion']
{'emotion', 'intensifier'} {'intensifier'} {'emotion'}
[]


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
good,0.1144,0.0227,0.0050,0.0415,0.1808,0.1835,0.1600,0.2921
kind,0.1233,0.0147,0.0110,0.0423,0.1819,0.1837,0.2017,0.2414
hostil,0.1027,0.1275,0.1993,0.2404,0.1017,0.1023,0.1148,0.0113
mean,0.1511,0.2292,0.1758,0.1050,0.0894,0.0866,0.0782,0.0846


...

intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
false,0.0001,0.0653,0.0573,0.0416,0.2143,0.1851,0.2182,0.2180
incorrect,0.0001,0.0412,0.0419,0.0225,0.2192,0.1979,0.2269,0.2504
correct,0.2852,0.2189,0.2176,0.2493,0.0006,0.0267,0.0017,0.0000
true,0.2499,0.1944,0.2046,0.2122,0.0406,0.0726,0.0257,0.0000


(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
24
Query time: 0.02041149139404297
Mean score unfiltered [-2.0..2.0]: 0.13006993511226028
Internal consistency (silhouette, correlation) for unfiltered: 0.8073816005812838
Internal consistency (Calinski&Harabasz)  for unfiltered: 7.404844842433426
Internal consistency (Davies&Bouldin) for unfiltered: 0.45632006856619434


index = ['emotion']
{'emotion', 'intensifier'} {'intensifier'} {'emotion'}
[]


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
false,0.3554,0.2194,0.1787,0.1691,0.0098,0.0650,0.0025,0.0001
incorrect,0.0157,0.2685,0.2850,0.2255,0.0597,0.1248,0.0207,0.0001
correct,0.0003,0.0436,0.0716,0.0952,0.2010,0.1598,0.2053,0.2232
true,0.0001,0.0036,0.0230,0.0468,0.2359,0.1698,0.2600,0.2608


(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
27
Query time: 0.02518463134765625
Mean score unfiltered [-2.0..2.0]: 0.04600714589469135
Internal consistency (silhouette, correlation) for unfiltered: 0.433796057711777
Internal consistency (Calinski&Harabasz)  for unfiltered: 2.0658672133532403
Internal consistency (Davies&Bouldin) for unfiltered: 0.8638373692231839


index = ['emotion']
{'emotion', 'intensifier'} {'intensifier'} {'emotion'}
[]


intensifier,never,very rarely,rarely,seldom,frequently,often,very frequently,always
emotion,,,,,,,,
false,0.3115,0.1229,0.1210,0.1189,0.0889,0.1047,0.1025,0.0297
incorrect,0.0310,0.1624,0.1637,0.1538,0.1342,0.1636,0.1475,0.0439
correct,0.0728,0.1118,0.1125,0.1280,0.1234,0.1120,0.1136,0.2259
true,0.0160,0.1120,0.1128,0.1061,0.1666,0.1347,0.1490,0.2028


# Load all questions

In [3]:
import sys, importlib
import qpsychometric
importlib.reload(sys.modules['qpsychometric'])
from qpsychometric.utils import load_questions

<module 'qpsychometric' from '/home/maorreu/qpsychometric/qpsychometric/__init__.py'>

In [4]:
load_questions()

load: phq_qmnli
load: gad_qmnli
load: soc_qmnli
load: big5_qmnli
load: sd3_qmnli
load: asi_qmnli


[qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli.PHQ9Q1,
 qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli.PHQ9Q2,
 qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli.PHQ9Q3,
 qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli.PHQ9Q4,
 qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli.PHQ9Q5,
 qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli.PHQ9Q6,
 qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli.PHQ9Q7,
 qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli.PHQ9Q8,
 qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli.PHQ9Q9,
 qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli.GAD7Q1,
 qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli.GAD7Q2,
 qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli.GAD7Q3,
 qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli.GAD7Q4,
 qpsychometr